# Nails

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2x0zwW4P54kBPyKWFAsAj1VdmY5_37CvA5hzRApq4UAcyK1vq")


In [ ]:
# ---- Imports ----
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import io, base64
from PIL import Image
import numpy as np
import joblib
import cv2
import torch
from torchvision import models, transforms
from torch import nn

# ---- Init ----
app = FastAPI()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# ---- Anemia Detection Model ----
model = joblib.load("/content/drive/My Drive/Grad/API/nails/random_forest_model.pkl")

def extract_features(image: Image.Image):
    image = image.resize((128, 128))
    image_np = np.array(image)

    if image_np.ndim == 2:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_GRAY2RGB)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(image_np, cv2.COLOR_BGR2HSV)

    hist = cv2.calcHist([hsv], [0, 1, 2], None, (8, 8, 8), [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist.reshape(1, -1)

@app.post("/predict-anemia")
async def predict_anemia(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
        features = extract_features(image)
        prediction = model.predict(features)[0]
        confidence = (
            np.max(model.predict_proba(features)) if hasattr(model, "predict_proba") else 1.0
        )

        return {
            "label": str(prediction),
            "confidence": round(float(confidence), 3)
        }
    except Exception as e:
        return {"error": str(e)}

# ---- Nail Segmentation Model ----
seg_model = models.segmentation.deeplabv3_resnet101(weights=None, aux_loss=True)
seg_model.classifier[4] = nn.Conv2d(256, 1, 1)
seg_model.aux_classifier[4] = nn.Conv2d(256, 1, 1)
seg_model.load_state_dict(torch.load("/content/drive/My Drive/Grad/API/nails/segmentation_nails.pth", map_location=device))
seg_model.to(device)
seg_model.eval()

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
raw_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

@app.post("/segment-nail")
async def segment_nail(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        img_pil = Image.open(io.BytesIO(contents)).convert("RGB")
        img_tensor = img_transform(img_pil).unsqueeze(0).to(device)
        raw_tensor = raw_transform(img_pil)

        with torch.no_grad():
            output = seg_model(img_tensor)['out']
            pred_mask = torch.sigmoid(output)
            pred_mask = (pred_mask > 0.5).float().cpu().squeeze(0)

        masked_rgb = raw_tensor * pred_mask
        masked_np = masked_rgb.permute(1, 2, 0).numpy()
        masked_np = np.clip(masked_np * 255, 0, 255).astype(np.uint8)
        masked_pil = Image.fromarray(masked_np)

        buffer = io.BytesIO()
        masked_pil.save(buffer, format="PNG")
        base64_img = base64.b64encode(buffer.getvalue()).decode("utf-8")

        return {"masked_image_base64": base64_img}
    except Exception as e:
        return {"error": str(e)}

# ---- Run the Server ----
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"🚀 Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [42966]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public URL: NgrokTunnel: "https://90e9-35-185-110-31.ngrok-free.app" -> "http://localhost:8000"


RuntimeError: Event loop stopped before Future completed.

# Palm

In [ ]:
!pip install fastapi[all]


In [ ]:
!pip install nest_asyncio pyngrok


In [ ]:
!ngrok config add-authtoken 2x0zwW4P54kBPyKWFAsAj1VdmY5_37CvA5hzRApq4UAcyK1vq


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import io, base64
from PIL import Image
import numpy as np
import cv2
import joblib
import tensorflow as tf

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

anemia_model = joblib.load("/content/drive/MyDrive/Grad/API/Palm/RF-balancedPalm.pkl")
segmentation_model = tf.keras.models.load_model(
    "/content/drive/MyDrive/Grad/API/Palm/palm_segmentation_model.h5",
    compile=False
)

In [ ]:


def extract_color_histogram(image: Image.Image):
    image = image.resize((128, 128))
    img_np = np.array(image)

    if img_np.ndim == 2:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(img_np, cv2.COLOR_RGB2HSV)

    hist = cv2.calcHist([hsv], [0, 1, 2], None, (8, 8, 8), [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist.reshape(1, -1)

@app.get("/")
def read_root():
    return {"message": "API is running. Try /segment-and-predict"}

@app.post("/segment-and-predict")
async def segment_and_predict(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")
        image_resized = image.resize((128, 128))
        image_np = np.array(image_resized) / 255.0

        pred_mask = segmentation_model.predict(np.expand_dims(image_np, axis=0))[0]
        mask = (pred_mask.squeeze() > 0.5).astype(np.uint8)

        masked_image = np.array(image_resized) * np.expand_dims(mask, axis=-1)
        masked_image = masked_image.astype(np.uint8)

        masked_pil = Image.fromarray(masked_image)
        features = extract_color_histogram(masked_pil)
        prediction = anemia_model.predict(features)[0]
        confidence = anemia_model.predict_proba(features)[0][prediction]

        buffer = io.BytesIO()
        masked_pil.save(buffer, format="PNG")
        base64_img = base64.b64encode(buffer.getvalue()).decode("utf-8")

        return {
            "label": str(prediction),
            "confidence": round(float(confidence), 3),
            "masked_image_base64": base64_img
        }
    except Exception as e:
        return {"error": str(e)}

nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)


# Conjunctiva

In [ ]:
!pip install fastapi[all]
!pip install nest_asyncio pyngrok

from google.colab import drive
drive.mount('/content/drive')

!ngrok config add-authtoken 2x0zwW4P54kBPyKWFAsAj1VdmY5_37CvA5hzRApq4UAcyK1vq

from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import io, base64
from PIL import Image
import numpy as np
import cv2
import joblib
import tensorflow as tf

# Initialize FastAPI app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load models
anemia_model_conjunctiva = joblib.load("/content/drive/MyDrive/Grad/API/Conjunctiva/xgboost_tunnig_model.pkl")
segmentation_model_conjunctiva = tf.keras.models.load_model(
    "/content/drive/MyDrive/Grad/API/Conjunctiva/conjunctiva_unet_best_model.h5",
    compile=False
)

# Function to extract color histogram features
def extract_color_histogram(image: Image.Image):
    image = image.resize((128, 128))
    img_np = np.array(image)

    if img_np.ndim == 2:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(img_np, cv2.COLOR_RGB2HSV)

    hist = cv2.calcHist([hsv], [0, 1, 2], None, (8, 8, 8), [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist.reshape(1, -1)

@app.get("/")
def read_root():
    return {"message": "Conjunctiva API is running. Try /segment-and-predict"}

@app.post("/segment-and-predict")
async def segment_and_predict(file: UploadFile = File(...)):
    try:
        # Read the uploaded image
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")
        image_resized = image.resize((128, 128))
        image_np = np.array(image_resized)

        # Convert RGB -> BGR
        image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        # Normalize
        image_bgr_norm = image_bgr.astype(np.float32) / 255.0

        # Apply segmentation model
        pred_mask = segmentation_model_conjunctiva.predict(np.expand_dims(image_bgr_norm, axis=0))[0]

        # Remove last dimension if exists
        if len(pred_mask.shape) == 3:
            pred_mask = pred_mask.squeeze(-1)

        # Threshold mask to obtain binary output
        mask = (pred_mask > 0.5).astype(np.uint8)

        # Apply mask to the image (RGB version for display)
        masked_image = image_np * np.expand_dims(mask, axis=-1)
        masked_image = masked_image.astype(np.uint8)

        masked_pil = Image.fromarray(masked_image)
        features = extract_color_histogram(masked_pil)

        # Make prediction
        prediction = anemia_model_conjunctiva.predict(features)[0]
        confidence = anemia_model_conjunctiva.predict_proba(features)[0][prediction]

        # Convert masked image to base64 string
        buffer = io.BytesIO()
        masked_pil.save(buffer, format="PNG")
        base64_img = base64.b64encode(buffer.getvalue()).decode("utf-8")

        return {
            "label": str(prediction),
            "confidence": round(float(confidence), 3),
            "masked_image_base64": base64_img
        }
    except Exception as e:
        return {"error": str(e)}

# Apply ngrok for public URL and run the FastAPI server
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)


# Three Model API

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok scikit-learn joblib pillow python-multipart


In [ ]:
!ngrok config add-authtoken 2x0zwW4P54kBPyKWFAsAj1VdmY5_37CvA5hzRApq4UAcyK1vq



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### new


In [ ]:
# Install packages if needed
# !pip install fastapi uvicorn nest-asyncio pyngrok scikit-learn joblib pillow python-multipart tensorflow opencv-python-headless

from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import io, base64
from PIL import Image
import numpy as np
import cv2
import joblib
import tensorflow as tf
import torch
from torchvision import models, transforms
from torch import nn
import traceback

# Initialize FastAPI
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Device for PyTorch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ----- Load All Models -----

# Nails Models
nail_rf_model = joblib.load("/content/drive/My Drive/Grad/API/nails/random_forest_model.pkl")
nail_seg_model = models.segmentation.deeplabv3_resnet101(weights=None, aux_loss=True)
nail_seg_model.classifier[4] = nn.Conv2d(256, 1, 1)
nail_seg_model.aux_classifier[4] = nn.Conv2d(256, 1, 1)
nail_seg_model.load_state_dict(torch.load("/content/drive/My Drive/Grad/API/nails/segmentation_nails.pth", map_location=device))
nail_seg_model.to(device).eval()

# Palm Models
palm_rf_model = joblib.load("/content/drive/MyDrive/Grad/API/Palm/RF-balancedPalm.pkl")
palm_seg_model = tf.keras.models.load_model("/content/drive/MyDrive/Grad/API/Palm/palm_segmentation_model.h5", compile=False)

# Conjunctiva Models
conj_rf_model = joblib.load("/content/drive/MyDrive/Grad/API/Conjunctiva/xgboost_model.pkl")
conj_seg_model = tf.keras.models.load_model("/content/drive/MyDrive/Grad/API/Conjunctiva/conjunctiva_unet_best_model.h5", compile=False)


# ----- Utility Functions -----

# Histogram extractor (common for all)
def extract_color_histogram(image: Image.Image):
    image = image.resize((128, 128))
    img_np = np.array(image)
    if img_np.ndim == 2:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(img_np, cv2.COLOR_RGB2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, (8, 8, 8), [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist.reshape(1, -1)

# Nails segmentation (Safe)
def segment_nail(image: Image.Image):
    image = image.convert("RGB").resize((224, 224))
    img_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    img_tensor = img_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = nail_seg_model(img_tensor)['out']
        pred_mask = torch.sigmoid(output)
        pred_mask = (pred_mask > 0.5).float().cpu().squeeze(0)
    raw_tensor = transforms.ToTensor()(image)
    masked_rgb = raw_tensor * pred_mask
    masked_np = masked_rgb.permute(1, 2, 0).numpy()
    masked_np = np.clip(masked_np * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(masked_np)

# Palm segmentation (Safe)
def segment_palm(image: Image.Image):
    image = image.convert("RGB").resize((128, 128))
    image_np = np.array(image) / 255.0
    pred_mask = palm_seg_model.predict(np.expand_dims(image_np, axis=0))[0]
    mask = (pred_mask.squeeze() > 0.5).astype(np.uint8)
    masked_image = np.array(image) * np.expand_dims(mask, axis=-1)
    return Image.fromarray(masked_image.astype(np.uint8))

# Conjunctiva segmentation (Safe)
def segment_conjunctiva(image: Image.Image):
    image = image.convert("RGB").resize((256, 256))
    image_np = np.array(image)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    image_bgr_norm = image_bgr.astype(np.float32) / 255.0
    pred_mask = conj_seg_model.predict(np.expand_dims(image_bgr_norm, axis=0))[0]
    if len(pred_mask.shape) == 3:
        pred_mask = pred_mask.squeeze(-1)
    mask = (pred_mask > 0.5).astype(np.uint8)
    masked_image = image_np * np.expand_dims(mask, axis=-1)
    return Image.fromarray(masked_image.astype(np.uint8))


# ----- Combine Results Logic -----
def combine_results(predictions: list):
    anemic_count = predictions.count(0)
    if anemic_count == 3:
        return "High likelihood Anemic"
    elif anemic_count == 2:
        return "Moderate likelihood Anemic"
    elif anemic_count == 1:
        return "Low likelihood Anemic"
    else:
        return "Healthy"


# ----- API Endpoint -----

@app.post("/predict")
async def predict_all(nail: UploadFile = File(...), palm: UploadFile = File(...), conjunctiva: UploadFile = File(...)):
    try:
        result = {}

        # ---- Nail ----
        try:
            nail_img = Image.open(io.BytesIO(await nail.read()))
            nail_masked = segment_nail(nail_img)
            nail_feat = extract_color_histogram(nail_masked)
            nail_pred = nail_rf_model.predict(nail_feat)[0]
            result['nail_prediction'] = int(nail_pred)
        except Exception as e:
            traceback.print_exc()
            result['nail_prediction'] = None

        # ---- Palm ----
        try:
            palm_img = Image.open(io.BytesIO(await palm.read()))
            palm_masked = segment_palm(palm_img)
            palm_feat = extract_color_histogram(palm_masked)
            palm_pred = palm_rf_model.predict(palm_feat)[0]
            result['palm_prediction'] = int(palm_pred)
        except Exception as e:
            traceback.print_exc()
            result['palm_prediction'] = None

        # ---- Conjunctiva ----
        try:
            conj_img = Image.open(io.BytesIO(await conjunctiva.read()))
            conj_masked = segment_conjunctiva(conj_img)
            conj_feat = extract_color_histogram(conj_masked)
            conj_pred = conj_rf_model.predict(conj_feat)[0]
            result['conjunctiva_prediction'] = int(conj_pred)
        except Exception as e:
            traceback.print_exc()
            result['conjunctiva_prediction'] = None

        # ---- Combine ----
        predictions = [
            result.get('nail_prediction'),
            result.get('palm_prediction'),
            result.get('conjunctiva_prediction')
        ]

        # fallback for combine_results to handle missing predictions
        safe_predictions = [p if p in [0, 1] else 1 for p in predictions]
        result['final_diagnosis'] = combine_results(safe_predictions)

        return result

    except Exception as e:
        traceback.print_exc()
        return {"error": str(e)}

# ----- Start server -----
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://593e-34-125-90-167.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [639]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
INFO:     154.177.143.76:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [639]
